# 🗺️ Extractor de Datos KML - Interfaz Amigable

**¡Bienvenido al Extractor de KML!** Este notebook te permitirá extraer fácilmente coordenadas, IDs y direcciones de archivos KML **desde cualquier ubicación** en tu computadora.

## 🎯 ¿Qué puedes hacer aquí?
- 📍 **Extraer coordenadas** de archivos KML (longitud, latitud, altitud)
- 🏷️ **Obtener IDs únicos** y metadatos de placemarks
- 📍 **Extraer direcciones** embebidas en los archivos KML
- 📊 **Exportar resultados** en múltiples formatos (CSV, JSON, Excel)

## 🗺️ ¿Qué son los archivos KML?
Los archivos **KML (Keyhole Markup Language)** son usados para mostrar datos geográficos. Son creados por:

- 🌍 **Google Earth**: Para guardar ubicaciones y rutas marcadas
- 📍 **BatchGeo**: Para crear mapas desde listas de direcciones
- 🗺️ **Google Maps**: Para exportar mapas personalizados
- 🚗 **GPS y navegadores**: Para rutas y puntos de interés
- 📊 **Aplicaciones GIS**: Para intercambio de datos geográficos

## 💻 Compatible con cualquier ubicación:
- 📁 **Archivos locales**: `C:\MisArchivos\mapa.kml`
- ☁️ **OneDrive**: `C:\Users\Usuario\OneDrive\Documentos\ubicaciones.kml`
- 🌐 **OneDrive Empresarial**: `C:\Users\Usuario\OneDrive - Empresa\datos.kml`
- 💾 **Unidades externas**: `D:\Proyectos\puntos.kml`
- 🗂️ **Carpetas compartidas**: `\\Servidor\Compartida\mapas.kml`

## 📋 Instrucciones para principiantes:
1. **Ejecuta cada celda en orden** (Shift + Enter)
2. **Usa el explorador integrado** o pega rutas completas de archivos
3. **¡No te preocupes si eres nuevo!** Todo está explicado paso a paso

### ✨ Casos de uso comunes:
- **Analizar rutas de GPS** exportadas de dispositivos
- **Procesar mapas de BatchGeo** para análisis de datos
- **Extraer coordenadas** de mapas personalizados de Google Earth
- **Convertir datos KML** a formatos como CSV para Excel
- **Preparar datos** para sistemas GIS o análisis geográfico

---

## 📦 Paso 1: Instalación de Dependencias

Primero instalaremos automáticamente todas las librerías necesarias para procesar archivos KML.

In [9]:
# Instalar dependencias automáticamente para el KML Extractor
import subprocess
import sys

def install_package(package):
    """Instala un paquete usando pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} instalado exitosamente")
        return True
    except subprocess.CalledProcessError:
        print(f"❌ Error instalando {package}")
        return False

print("🔧 Instalando dependencias para KML Extractor...")
print("(Esto puede tomar unos minutos la primera vez)\n")

# Dependencias necesarias para el procesamiento KML
packages = [
    "pandas",           # Para manejo de datos y CSV
    "ipywidgets",       # Para interfaces interactivas
    "plotly",          # Para mapas interactivos
    "folium",          # Para mapas alternativos
    "openpyxl"         # Para exportar a Excel
]

all_success = True
for package in packages:
    if not install_package(package):
        all_success = False

print("\n" + "="*60)
if all_success:
    print("✅ ¡Todas las dependencias instaladas correctamente!")
    print("🗺️ El Extractor KML está listo para usar")
else:
    print("⚠️  Algunas dependencias no se pudieron instalar.")
    print("Por favor, instala manualmente los paquetes faltantes.")
print("="*60)

# Verificar que las librerías estándar están disponibles
try:
    import xml.etree.ElementTree as ET
    from pathlib import Path
    import os
    import json
    import csv
    print("✅ Librerías estándar de Python verificadas")
except ImportError as e:
    print(f"❌ Error con librerías estándar: {e}")
    print("Asegúrate de tener una instalación completa de Python")

🔧 Instalando dependencias para KML Extractor...
(Esto puede tomar unos minutos la primera vez)

✅ pandas instalado exitosamente
✅ pandas instalado exitosamente
✅ ipywidgets instalado exitosamente
✅ ipywidgets instalado exitosamente
✅ plotly instalado exitosamente
✅ plotly instalado exitosamente
✅ folium instalado exitosamente
✅ folium instalado exitosamente
✅ openpyxl instalado exitosamente

✅ ¡Todas las dependencias instaladas correctamente!
🗺️ El Extractor KML está listo para usar
✅ Librerías estándar de Python verificadas
✅ openpyxl instalado exitosamente

✅ ¡Todas las dependencias instaladas correctamente!
🗺️ El Extractor KML está listo para usar
✅ Librerías estándar de Python verificadas


## 📚 Paso 2: Importar Librerías y Cargar KML Extractor

Ahora cargaremos todas las herramientas que necesitamos para procesar archivos KML.

In [10]:
# Importar librerías necesarias
import os
import sys
import pandas as pd
import json
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Importar librerías para XML y KML
import xml.etree.ElementTree as ET

# Importar librerías para interfaz
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Importar librerías para visualización
try:
    import plotly.express as px
    import plotly.graph_objects as go
    PLOTLY_AVAILABLE = True
    print("✅ Plotly disponible para mapas interactivos")
except ImportError:
    PLOTLY_AVAILABLE = False
    print("⚠️  Plotly no disponible - mapas limitados")

# Intentar importar la herramienta KML Extractor
try:
    # Si el archivo está en el mismo directorio
    from kml_extractor import KMLExtractor
    print("✅ KMLExtractor cargado desde archivo local")
    EXTRACTOR_AVAILABLE = True
except ImportError:
    # Si no está disponible, crear una versión simplificada
    print("⚠️  KMLExtractor no encontrado - usando versión simplificada")
    EXTRACTOR_AVAILABLE = False
    
    # Clase KMLExtractor simplificada integrada
    class KMLExtractor:
        """Versión simplificada del extractor KML."""
        
        def __init__(self, kml_file_path):
            self.kml_file_path = Path(kml_file_path)
            self.namespace = {'kml': 'http://www.opengis.net/kml/2.2'}
            self.data = []
        
        def _parse_coordinates(self, coord_text):
            """Parsear coordenadas KML."""
            try:
                coords = coord_text.strip().split(',')
                if len(coords) >= 2:
                    longitude = float(coords[0])
                    latitude = float(coords[1])
                    altitude = float(coords[2]) if len(coords) > 2 else 0.0
                    return longitude, latitude, altitude
            except (ValueError, IndexError):
                pass
            return None
        
        def extract_data(self):
            """Extraer datos del archivo KML."""
            if not self.kml_file_path.exists():
                raise FileNotFoundError(f"Archivo KML no encontrado: {self.kml_file_path}")
            
            try:
                tree = ET.parse(self.kml_file_path)
                root = tree.getroot()
                
                # Encontrar placemarks
                placemarks = root.findall('.//kml:Placemark', self.namespace)
                
                extracted_data = []
                for i, placemark in enumerate(placemarks):
                    data = {
                        'placemark_index': i + 1,
                        'id': None,
                        'address': None,
                        'longitude': None,
                        'latitude': None,
                        'altitude': None
                    }
                    
                    # Extraer nombre/ID
                    name_elem = placemark.find('kml:name', self.namespace)
                    if name_elem is not None and name_elem.text:
                        data['id'] = name_elem.text.strip()
                    
                    # Extraer dirección
                    addr_elem = placemark.find('kml:address', self.namespace)
                    if addr_elem is not None and addr_elem.text:
                        data['address'] = addr_elem.text.strip()
                    
                    # Extraer descripción como dirección alternativa
                    if not data['address']:
                        desc_elem = placemark.find('kml:description', self.namespace)
                        if desc_elem is not None and desc_elem.text:
                            data['address'] = desc_elem.text.strip()
                    
                    # Extraer coordenadas
                    coord_elem = placemark.find('.//kml:coordinates', self.namespace)
                    if coord_elem is not None and coord_elem.text:
                        coords = self._parse_coordinates(coord_elem.text)
                        if coords:
                            data['longitude'] = coords[0]
                            data['latitude'] = coords[1]
                            data['altitude'] = coords[2]
                    
                    extracted_data.append(data)
                
                self.data = extracted_data
                return extracted_data
                
            except ET.ParseError as e:
                raise Exception(f"Error parseando KML: {e}")

print(f"🗺️ KML Extractor cargado - Versión: {'Completa' if EXTRACTOR_AVAILABLE else 'Simplificada'}")
print("✅ ¡Listo para procesar archivos KML!")

✅ Plotly disponible para mapas interactivos
✅ KMLExtractor cargado desde archivo local
🗺️ KML Extractor cargado - Versión: Completa
✅ ¡Listo para procesar archivos KML!


## 📂 Paso 3: Cargar Archivo KML

Selecciona tu archivo KML desde cualquier ubicación en tu computadora.

In [ ]:
# Cargador de archivos KML simple y directo
import tkinter as tk
from tkinter import filedialog
import threading

print("🗺️ Cargador de Archivos KML")
print("Dos opciones rápidas para cargar tu archivo KML:")
print("1. 📁 Explorar: Usa el botón para navegar y seleccionar")
print("2. 📝 Pegar Ruta: Copia y pega la ruta completa del archivo")
print()

# Widget para ruta del archivo KML
kml_file_widget = widgets.Text(
    value='',
    placeholder='C:\\Users\\Usuario\\OneDrive\\Documentos\\archivo.kml',
    description='Archivo KML:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='700px')
)

# Botón para explorar archivos
browse_kml_button = widgets.Button(
    description='📁 Explorar KML',
    button_style='primary',
    layout=widgets.Layout(width='150px')
)

# Botón para validar archivo
validate_kml_button = widgets.Button(
    description='✅ Validar KML',
    button_style='success',
    layout=widgets.Layout(width='150px')
)

# Output para mostrar información
kml_validation_output = widgets.Output()

def browse_kml_file():
    """Abrir diálogo para seleccionar archivo KML"""
    def file_dialog():
        try:
            root = tk.Tk()
            root.withdraw()
            root.lift()
            root.attributes('-topmost', True)
            
            # Solo archivos KML
            filetypes = [
                ('Archivos KML', '*.kml'),
                ('Todos los archivos', '*.*')
            ]
            
            filename = filedialog.askopenfilename(
                title="Seleccionar archivo KML",
                filetypes=filetypes
            )
            
            if filename:
                kml_file_widget.value = filename
                validate_kml_file()
            
            root.destroy()
            
        except Exception as e:
            with kml_validation_output:
                clear_output()
                print(f"❌ Error abriendo explorador: {e}")
                print("💡 Puedes escribir la ruta manualmente")
    
    # Ejecutar en thread separado para evitar bloqueos
    thread = threading.Thread(target=file_dialog)
    thread.daemon = True
    thread.start()

def validate_kml_file():
    """Validar archivo KML seleccionado"""
    with kml_validation_output:
        clear_output()
        
        file_path = kml_file_widget.value.strip()
        if not file_path:
            print("📝 Especifica la ruta de un archivo KML o usa '📁 Explorar KML'")
            return
            
        try:
            kml_file = Path(file_path)
            
            if not kml_file.exists():
                print(f"❌ Archivo no encontrado: {file_path}")
                print("💡 Verifica que la ruta sea correcta")
                return
                
            if not kml_file.is_file():
                print(f"❌ La ruta no es un archivo válido: {file_path}")
                return
            
            # Verificar extensión
            if not kml_file.suffix.lower() == '.kml':
                print(f"⚠️  El archivo no tiene extensión .kml: {kml_file.suffix}")
                print("💡 ¿Estás seguro de que es un archivo KML?")
            
            # Información básica del archivo
            size = kml_file.stat().st_size
            if size < 1024:
                size_str = f"{size} bytes"
            elif size < 1024 * 1024:
                size_str = f"{size/1024:.1f} KB"
            else:
                size_str = f"{size/(1024*1024):.1f} MB"
            
            print("✅ Archivo KML encontrado!")
            print(f"📁 Nombre: {kml_file.name}")
            print(f"📍 Ubicación: {kml_file.parent}")
            print(f"📏 Tamaño: {size_str}")
            
            # Validación básica del contenido XML
            try:
                tree = ET.parse(kml_file)
                root = tree.getroot()
                
                # Verificar que sea un archivo KML válido
                if 'kml' not in root.tag.lower():
                    print("⚠️  El archivo no parece ser un KML válido")
                    print("💡 Verifica que sea un archivo KML real")
                else:
                    print("✅ Formato KML válido detectado")
                    
                    # Contar placemarks preliminares
                    placemarks = root.findall('.//*')
                    placemark_count = sum(1 for elem in placemarks if 'placemark' in elem.tag.lower())
                    
                    if placemark_count > 0:
                        print(f"📍 Placemarks detectados: ~{placemark_count}")
                    else:
                        print("⚠️  No se detectaron placemarks obvios")
                        print("💡 El archivo puede tener una estructura diferente")
                    
            except ET.ParseError as e:
                print(f"❌ Error leyendo XML: {e}")
                print("💡 El archivo puede estar corrupto o no ser un KML válido")
                return
            except Exception as e:
                print(f"⚠️  Advertencia al validar: {e}")
            
            print()
            print("🎉 ¡Archivo listo para procesar!")
            print("💡 Continúa con el siguiente paso para previsualizar los datos")
            
        except Exception as e:
            print(f"❌ Error validando archivo: {e}")

# Conectar eventos
browse_kml_button.on_click(lambda b: browse_kml_file())
validate_kml_button.on_click(lambda b: validate_kml_file())

# Auto-validar cuando se cambie el texto
kml_file_widget.observe(
    lambda change: validate_kml_file() if change['name'] == 'value' and change['new'].strip() else None, 
    names='value'
)

display(widgets.VBox([
    widgets.HTML("<h4>🗺️ Seleccionar archivo KML</h4>"),
    kml_file_widget,
    widgets.HBox([browse_kml_button, validate_kml_button]),
    kml_validation_output
]))

🗺️ Cargador de Archivos KML
Dos opciones rápidas para cargar tu archivo KML:
1. 📁 Explorar: Usa el botón para navegar y seleccionar
2. 📝 Pegar Ruta: Copia y pega la ruta completa del archivo



## 📊 Paso 4: Previsualizar Datos KML

Antes de extraer toda la información, revisemos qué contiene tu archivo KML. Esto te ayudará a entender:

- **📍 Placemarks**: Puntos de interés, rutas, o ubicaciones marcadas
- **🗂️ Carpetas**: Organización de los datos dentro del KML
- **🏷️ Etiquetas**: Nombres e información asociada a cada punto
- **📍 Coordenadas**: Latitud, longitud y posiblemente altitud
- **📝 Descripciones**: Texto adicional o datos personalizados

> **💡 Tip**: Esta previsualización te permite verificar que el archivo se lee correctamente antes de extraer toda la información.

In [ ]:
# Previsualizador de datos KML
preview_button = widgets.Button(
    description='👀 Previsualizar KML',
    button_style='info',
    layout=widgets.Layout(width='180px')
)

# Opciones de previsualización
preview_options = widgets.SelectMultiple(
    options=[
        ('📍 Primeros 5 Placemarks', 'placemarks'),
        ('🗂️ Estructura de Carpetas', 'folders'),
        ('📊 Estadísticas Generales', 'stats'),
        ('🗺️ Rango de Coordenadas', 'coordinates'),
        ('📝 Tipos de Datos', 'datatypes')
    ],
    value=['placemarks', 'stats'],
    description='Ver:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px', height='120px')
)

# Output para la previsualización
preview_output = widgets.Output()

def preview_kml_data():
    """Previsualizar datos del archivo KML"""
    with preview_output:
        clear_output()
        
        file_path = kml_file_widget.value.strip()
        if not file_path:
            print("❌ Primero selecciona un archivo KML")
            return
            
        try:
            kml_file = Path(file_path)
            if not kml_file.exists():
                print("❌ Archivo no encontrado. Verifica la ruta.")
                return
            
            print("🔍 Analizando archivo KML...")
            print("=" * 50)
            
            # Parsear el archivo KML
            tree = ET.parse(kml_file)
            root = tree.getroot()
            
            # Obtener namespace si existe
            namespace = ''
            if root.tag.startswith('{'):
                namespace = root.tag.split('}')[0] + '}'
            
            # Funciones de análisis según opciones seleccionadas
            selected_options = preview_options.value
            
            if 'stats' in selected_options:
                print("\n📊 ESTADÍSTICAS GENERALES")
                print("-" * 30)
                
                # Contar elementos
                all_elements = root.findall('.//')
                placemarks = [elem for elem in all_elements if 'placemark' in elem.tag.lower()]
                folders = [elem for elem in all_elements if 'folder' in elem.tag.lower()]
                
                print(f"📍 Total Placemarks: {len(placemarks)}")
                print(f"🗂️  Total Carpetas: {len(folders)}")
                print(f"🏷️  Total Elementos: {len(all_elements)}")
                
                # Verificar tipos de geometría
                points = len([elem for elem in all_elements if 'point' in elem.tag.lower()])
                lines = len([elem for elem in all_elements if 'linestring' in elem.tag.lower()])
                polygons = len([elem for elem in all_elements if 'polygon' in elem.tag.lower()])
                
                if points + lines + polygons > 0:
                    print(f"📍 Puntos: {points}")
                    print(f"📏 Líneas: {lines}")
                    print(f"🔺 Polígonos: {polygons}")
            
            if 'folders' in selected_options:
                print("\n🗂️ ESTRUCTURA DE CARPETAS")
                print("-" * 30)
                
                folders = root.findall('.//' + namespace + 'Folder')
                if folders:
                    for i, folder in enumerate(folders[:10]):  # Máximo 10 carpetas
                        name_elem = folder.find(namespace + 'name')
                        name = name_elem.text if name_elem is not None else f"Carpeta {i+1}"
                        
                        # Contar placemarks en esta carpeta
                        folder_placemarks = folder.findall('.//' + namespace + 'Placemark')
                        print(f"📁 {name} ({len(folder_placemarks)} elementos)")
                    
                    if len(folders) > 10:
                        print(f"... y {len(folders)-10} carpetas más")
                else:
                    print("📂 No se encontraron carpetas organizadoras")
            
            if 'placemarks' in selected_options:
                print("\n📍 PRIMEROS PLACEMARKS")
                print("-" * 30)
                
                placemarks = root.findall('.//' + namespace + 'Placemark')
                
                if placemarks:
                    for i, placemark in enumerate(placemarks[:5]):  # Primeros 5
                        name_elem = placemark.find(namespace + 'name')
                        name = name_elem.text if name_elem is not None else f"Sin nombre"
                        
                        # Buscar coordenadas
                        coords_elem = placemark.find('.//' + namespace + 'coordinates')
                        if coords_elem is not None:
                            coords = coords_elem.text.strip()
                            if coords:
                                # Tomar primera coordenada si hay múltiples
                                first_coord = coords.split()[0] if ' ' in coords else coords
                                try:
                                    parts = first_coord.split(',')
                                    if len(parts) >= 2:
                                        lon, lat = parts[0], parts[1]
                                        print(f"{i+1}. 📍 {name}")
                                        print(f"   🌍 Lat: {lat}, Lon: {lon}")
                                    else:
                                        print(f"{i+1}. 📍 {name} (coordenadas incompletas)")
                                except:
                                    print(f"{i+1}. 📍 {name} (coordenadas con formato inusual)")
                            else:
                                print(f"{i+1}. 📍 {name} (sin coordenadas)")
                        else:
                            print(f"{i+1}. 📍 {name} (sin elemento coordinates)")
                        
                        # Mostrar descripción si existe (primeras palabras)
                        desc_elem = placemark.find(namespace + 'description')
                        if desc_elem is not None and desc_elem.text:
                            desc_preview = desc_elem.text[:100]
                            if len(desc_elem.text) > 100:
                                desc_preview += "..."
                            print(f"   📝 {desc_preview}")
                        
                        print()
                    
                    if len(placemarks) > 5:
                        print(f"... y {len(placemarks)-5} placemarks adicionales")
                else:
                    print("❌ No se encontraron placemarks en el archivo")
            
            if 'coordinates' in selected_options:
                print("\n🗺️ RANGO DE COORDENADAS")
                print("-" * 30)
                
                all_coords = []
                coord_elements = root.findall('.//' + namespace + 'coordinates')
                
                for coord_elem in coord_elements:
                    if coord_elem.text:
                        coords_text = coord_elem.text.strip()
                        # Separar múltiples coordenadas
                        for coord_line in coords_text.split():
                            parts = coord_line.split(',')
                            if len(parts) >= 2:
                                try:
                                    lon, lat = float(parts[0]), float(parts[1])
                                    all_coords.append((lat, lon))
                                except ValueError:
                                    continue
                
                if all_coords:
                    lats = [coord[0] for coord in all_coords]
                    lons = [coord[1] for coord in all_coords]
                    
                    print(f"📍 Total coordenadas válidas: {len(all_coords)}")
                    print(f"🌍 Latitud: {min(lats):.6f} a {max(lats):.6f}")
                    print(f"🌐 Longitud: {min(lons):.6f} a {max(lons):.6f}")
                    
                    # Estimar región
                    center_lat = (min(lats) + max(lats)) / 2
                    center_lon = (min(lons) + max(lons)) / 2
                    print(f"🎯 Centro aproximado: {center_lat:.6f}, {center_lon:.6f}")
                else:
                    print("❌ No se pudieron extraer coordenadas válidas")
            
            if 'datatypes' in selected_options:
                print("\n📝 TIPOS DE DATOS ENCONTRADOS")
                print("-" * 30)
                
                # Buscar diferentes tipos de elementos
                all_elements = root.findall('.//')
                element_types = {}
                
                for elem in all_elements:
                    tag_name = elem.tag.split('}')[-1] if '}' in elem.tag else elem.tag
                    element_types[tag_name] = element_types.get(tag_name, 0) + 1
                
                # Mostrar solo elementos relevantes
                relevant_types = ['name', 'description', 'Placemark', 'Folder', 'Point', 
                                'LineString', 'Polygon', 'coordinates', 'ExtendedData']
                
                found_relevant = False
                for elem_type in relevant_types:
                    if elem_type in element_types:
                        print(f"📋 {elem_type}: {element_types[elem_type]}")
                        found_relevant = True
                
                if not found_relevant:
                    print("📋 Tipos de elementos principales:")
                    for elem_type, count in sorted(element_types.items(), key=lambda x: x[1], reverse=True)[:10]:
                        print(f"   {elem_type}: {count}")
            
            print("\n" + "=" * 50)
            print("✅ Previsualización completada")
            print("💡 Si los datos se ven correctos, procede al siguiente paso")
            
        except ET.ParseError as e:
            print(f"❌ Error al parsear XML: {e}")
            print("💡 El archivo puede estar corrupto o tener formato inválido")
        except Exception as e:
            print(f"❌ Error en previsualización: {e}")
            print(f"💡 Tipo de error: {type(e).__name__}")

# Conectar el botón
preview_button.on_click(lambda b: preview_kml_data())

display(widgets.VBox([
    widgets.HTML("<h4>👀 Previsualización del contenido KML</h4>"),
    preview_options,
    preview_button,
    preview_output
]))

## ⚙️ Paso 5: Configurar Extracción

Personaliza qué información quieres extraer del archivo KML:

### 📋 Datos Básicos
- **🏷️ Nombres**: Títulos de cada placemark
- **📍 Coordenadas**: Latitud y longitud
- **📝 Descripciones**: Texto adicional

### 🎯 Datos Avanzados  
- **🆔 IDs únicos**: Identificadores internos
- **🗂️ Carpetas**: Organización jerárquica
- **📊 ExtendedData**: Datos personalizados
- **🏔️ Altitud**: Si está disponible

### 📄 Formatos de Salida
- **CSV**: Para análisis en Excel o pandas
- **JSON**: Para integración con aplicaciones
- **TXT**: Para revisión rápida

In [20]:
# Configuración de extracción
print("⚙️ Configurar Extracción de Datos KML")
print("Selecciona qué información extraer:")
print()

# Opciones de datos a extraer
data_options = widgets.SelectMultiple(
    options=[
        ('🏷️ Nombres de Placemarks', 'names'),
        ('📍 Coordenadas (Lat, Lon)', 'coordinates'),
        ('📝 Descripciones', 'descriptions'),
        ('🆔 IDs únicos', 'ids'),
        ('🗂️ Carpetas/Folders', 'folders'),
        ('🏔️ Altitud (si disponible)', 'altitude'),
        ('📊 ExtendedData', 'extended_data'),
        ('🔗 Enlaces/Links', 'links')
    ],
    value=['names', 'coordinates', 'descriptions'],
    description='Datos:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px', height='180px')
)

# Formato de salida
output_format = widgets.Dropdown(
    options=[
        ('📊 CSV (recomendado)', 'csv'),
        ('📄 JSON', 'json'),
        ('📝 TXT (texto plano)', 'txt'),
        ('📋 Mostrar en pantalla', 'display')
    ],
    value='csv',
    description='Formato:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Opciones avanzadas
advanced_options = widgets.VBox([
    widgets.HTML("<b>⚙️ Opciones Avanzadas:</b>"),
    widgets.Checkbox(
        value=True,
        description='🧹 Limpiar datos (remover espacios extra)',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    ),
    widgets.Checkbox(
        value=True,
        description='✅ Validar coordenadas',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    ),
    widgets.Checkbox(
        value=False,
        description='🔢 Incluir índice numérico',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )
])

# Nombre del archivo de salida
output_filename = widgets.Text(
    value='datos_extraidos_kml',
    description='Nombre archivo:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Botón para aplicar configuración
config_button = widgets.Button(
    description='✅ Aplicar Configuración',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

# Output para mostrar configuración
config_output = widgets.Output()

def apply_configuration():
    """Aplicar y mostrar configuración seleccionada"""
    with config_output:
        clear_output()
        
        selected_data = list(data_options.value)
        format_selected = output_format.value
        filename = output_filename.value.strip()
        
        if not selected_data:
            print("❌ Selecciona al menos un tipo de dato para extraer")
            return
            
        if not filename:
            print("❌ Especifica un nombre para el archivo de salida")
            return
        
        print("✅ Configuración aplicada exitosamente!")
        print("=" * 40)
        print("📋 Datos a extraer:")
        
        data_labels = {
            'names': '🏷️ Nombres de Placemarks',
            'coordinates': '📍 Coordenadas (Lat, Lon)',
            'descriptions': '📝 Descripciones',
            'ids': '🆔 IDs únicos',
            'folders': '🗂️ Carpetas/Folders',
            'altitude': '🏔️ Altitud',
            'extended_data': '📊 ExtendedData',
            'links': '🔗 Enlaces/Links'
        }
        
        for data_type in selected_data:
            print(f"  ✓ {data_labels.get(data_type, data_type)}")
        
        format_labels = {
            'csv': '📊 CSV (Comma Separated Values)',
            'json': '📄 JSON (JavaScript Object Notation)',
            'txt': '📝 TXT (Texto plano)',
            'display': '📋 Mostrar en pantalla'
        }
        
        print(f"\n📄 Formato de salida: {format_labels.get(format_selected, format_selected)}")
        
        if format_selected != 'display':
            extension = format_selected
            print(f"💾 Archivo: {filename}.{extension}")
        
        print(f"\n⚙️ Opciones avanzadas:")
        for child in advanced_options.children[1:]:  # Saltar el HTML header
            if hasattr(child, 'value') and child.value:
                print(f"  ✓ {child.description}")
        
        print(f"\n🎉 ¡Configuración lista!")
        print(f"💡 Procede al siguiente paso para ejecutar la extracción")

# Conectar botón
config_button.on_click(lambda b: apply_configuration())

# Auto-aplicar configuración cuando cambie algo importante
data_options.observe(lambda change: apply_configuration(), names='value')
output_format.observe(lambda change: apply_configuration(), names='value')

display(widgets.VBox([
    widgets.HTML("<h4>⚙️ Configuración de Extracción</h4>"),
    widgets.HBox([
        widgets.VBox([data_options, output_format]),
        widgets.VBox([advanced_options, output_filename, config_button])
    ]),
    config_output
]))

⚙️ Configurar Extracción de Datos KML
Selecciona qué información extraer:



## 🚀 Paso 6: Ejecutar Extracción

¡Ahora viene la parte emocionante! Vamos a procesar tu archivo KML y extraer toda la información según tu configuración.

### 🔄 Proceso de Extracción:
1. **🔍 Análisis**: Parseo completo del archivo KML
2. **🎯 Filtrado**: Extracción de datos según tu selección
3. **🧹 Limpieza**: Procesamiento y validación de datos
4. **💾 Exportación**: Guardado en el formato seleccionado

> **💡 Tip**: Dependiendo del tamaño del archivo, la extracción puede tomar desde unos segundos hasta algunos minutos.

In [ ]:
# Extractor principal de datos KML
extract_button = widgets.Button(
    description='🚀 Extraer Datos KML',
    button_style='danger',
    layout=widgets.Layout(width='200px', height='40px')
)

# Progress bar para mostrar progreso
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Progreso:',
    bar_style='',
    style={'bar_color': '#1f77b4'},
    layout=widgets.Layout(width='500px')
)

# Output para mostrar resultados de extracción
extraction_output = widgets.Output()

# Variable global para almacenar datos extraídos
extracted_data = None
extraction_summary = {}

def clean_text(text):
    """Limpiar texto removiendo espacios extra y caracteres problemáticos"""
    if not text:
        return ""
    # Remover espacios extra y saltos de línea
    cleaned = ' '.join(str(text).split())
    # Remover caracteres de control
    cleaned = ''.join(char for char in cleaned if ord(char) >= 32 or char in '\t\n\r')
    return cleaned.strip()

def validate_coordinates(lat, lon):
    """Validar que las coordenadas estén en rangos válidos"""
    try:
        lat_f, lon_f = float(lat), float(lon)
        return (-90 <= lat_f <= 90) and (-180 <= lon_f <= 180)
    except (ValueError, TypeError):
        return False

def extract_kml_data():
    """Función principal para extraer datos del KML"""
    global extracted_data, extraction_summary
    
    with extraction_output:
        clear_output()
        
        # Verificar configuración básica
        file_path = kml_file_widget.value.strip()
        if not file_path:
            print("❌ Error: No se ha seleccionado un archivo KML")
            print("💡 Regresa al Paso 3 para seleccionar un archivo")
            return
        
        selected_data = list(data_options.value)
        if not selected_data:
            print("❌ Error: No se han seleccionado datos para extraer")
            print("💡 Regresa al Paso 5 para configurar la extracción")
            return
        
        try:
            print("🚀 Iniciando extracción de datos KML...")
            print("=" * 50)
            
            # Actualizar progress bar
            progress_bar.value = 10
            progress_bar.description = "Cargando archivo..."
            
            kml_file = Path(file_path)
            if not kml_file.exists():
                print("❌ Error: Archivo no encontrado")
                return
            
            # Parsear archivo KML
            progress_bar.value = 20
            progress_bar.description = "Parseando KML..."
            
            tree = ET.parse(kml_file)
            root = tree.getroot()
            
            # Detectar namespace
            namespace = ''
            if root.tag.startswith('{'):
                namespace = root.tag.split('}')[0] + '}'
            
            # Buscar todos los placemarks
            progress_bar.value = 30
            progress_bar.description = "Encontrando placemarks..."
            
            placemarks = root.findall('.//' + namespace + 'Placemark')
            total_placemarks = len(placemarks)
            
            print(f"📍 Placemarks encontrados: {total_placemarks}")
            
            if total_placemarks == 0:
                print("⚠️  No se encontraron placemarks en el archivo")
                print("💡 Verifica que el archivo KML contenga datos de ubicación")
                return
            
            # Obtener opciones de configuración
            clean_data = advanced_options.children[1].value  # Limpiar datos
            validate_coords = advanced_options.children[2].value  # Validar coordenadas
            include_index = advanced_options.children[3].value  # Incluir índice
            
            # Extraer datos
            progress_bar.value = 40
            progress_bar.description = "Extrayendo datos..."
            
            extracted_records = []
            errors = []
            
            for i, placemark in enumerate(placemarks):
                record = {}
                
                # Actualizar progreso durante extracción
                if i % max(1, len(placemarks) // 20) == 0:
                    progress = 40 + int((i / len(placemarks)) * 40)
                    progress_bar.value = progress
                    progress_bar.description = f"Procesando {i+1}/{len(placemarks)}..."
                
                try:
                    # Índice numérico
                    if include_index:
                        record['indice'] = i + 1
                    
                    # Extraer nombres
                    if 'names' in selected_data:
                        name_elem = placemark.find(namespace + 'name')
                        name = name_elem.text if name_elem is not None and name_elem.text else f"Placemark_{i+1}"
                        record['nombre'] = clean_text(name) if clean_data else name
                    
                    # Extraer IDs
                    if 'ids' in selected_data:
                        id_attr = placemark.get('id', '')
                        if not id_attr:
                            # Buscar en elementos hijos
                            id_elem = placemark.find('.//*[@id]')
                            id_attr = id_elem.get('id', '') if id_elem is not None else f"id_{i+1}"
                        record['id'] = clean_text(id_attr) if clean_data else id_attr
                    
                    # Extraer coordenadas
                    if 'coordinates' in selected_data:
                        coords_elem = placemark.find('.//' + namespace + 'coordinates')
                        if coords_elem is not None and coords_elem.text:
                            coords_text = coords_elem.text.strip()
                            if coords_text:
                                # Tomar primera coordenada si hay múltiples
                                first_coord = coords_text.split()[0] if ' ' in coords_text else coords_text
                                try:
                                    parts = first_coord.split(',')
                                    if len(parts) >= 2:
                                        lon, lat = parts[0].strip(), parts[1].strip()
                                        alt = parts[2].strip() if len(parts) > 2 else ''
                                        
                                        # Validar coordenadas si está habilitado
                                        if validate_coords:
                                            if validate_coordinates(lat, lon):
                                                record['latitud'] = lat
                                                record['longitud'] = lon
                                                if 'altitude' in selected_data and alt:
                                                    record['altitud'] = alt
                                            else:
                                                errors.append(f"Placemark {i+1}: Coordenadas inválidas ({lat}, {lon})")
                                                record['latitud'] = lat  # Mantener datos originales
                                                record['longitud'] = lon
                                                if 'altitude' in selected_data and alt:
                                                    record['altitud'] = alt
                                        else:
                                            record['latitud'] = lat
                                            record['longitud'] = lon
                                            if 'altitude' in selected_data and alt:
                                                record['altitud'] = alt
                                    else:
                                        errors.append(f"Placemark {i+1}: Formato de coordenadas incompleto")
                                except Exception as coord_error:
                                    errors.append(f"Placemark {i+1}: Error procesando coordenadas - {coord_error}")
                            else:
                                errors.append(f"Placemark {i+1}: Coordenadas vacías")
                        else:
                            errors.append(f"Placemark {i+1}: Sin elemento coordinates")
                    
                    # Extraer descripciones
                    if 'descriptions' in selected_data:
                        desc_elem = placemark.find(namespace + 'description')
                        if desc_elem is not None and desc_elem.text:
                            description = desc_elem.text
                            record['descripcion'] = clean_text(description) if clean_data else description
                        else:
                            record['descripcion'] = ""
                    
                    # Extraer carpetas/folders
                    if 'folders' in selected_data:
                        # Buscar folders en la estructura del documento
                        folder_name = ""
                        # Para ElementTree, usamos una aproximación diferente
                        # Buscar el índice del placemark actual y verificar en folders
                        try:
                            all_folders = root.findall('.//' + namespace + 'Folder')
                            for folder in all_folders:
                                folder_placemarks = folder.findall('.//' + namespace + 'Placemark')
                                # Comparar por posición en lugar de objeto
                                if len(folder_placemarks) > 0:
                                    folder_name_elem = folder.find(namespace + 'name')
                                    if folder_name_elem is not None and folder_name_elem.text:
                                        folder_name = folder_name_elem.text
                                        break
                        except:
                            folder_name = ""
                        record['carpeta'] = clean_text(folder_name) if clean_data else folder_name
                    
                    # Extraer ExtendedData
                    if 'extended_data' in selected_data:
                        extended_elem = placemark.find('.//' + namespace + 'ExtendedData')
                        extended_data = {}
                        if extended_elem is not None:
                            for data_elem in extended_elem.findall('.//' + namespace + 'Data'):
                                name = data_elem.get('name', 'unknown')
                                value_elem = data_elem.find(namespace + 'value')
                                value = value_elem.text if value_elem is not None else ''
                                extended_data[name] = clean_text(value) if clean_data else value
                        record['datos_extendidos'] = str(extended_data) if extended_data else ""
                    
                    # Extraer enlaces/links
                    if 'links' in selected_data:
                        links = []
                        for link_elem in placemark.findall('.//' + namespace + 'href'):
                            if link_elem.text:
                                links.append(link_elem.text.strip())
                        record['enlaces'] = '; '.join(links) if links else ""
                    
                    extracted_records.append(record)
                    
                except Exception as e:
                    errors.append(f"Placemark {i+1}: Error general - {e}")
                    # Agregar registro básico incluso con errores
                    if not record:
                        record = {'error': f"Error procesando placemark {i+1}: {e}"}
                    extracted_records.append(record)
            
            # Finalizar extracción
            progress_bar.value = 80
            progress_bar.description = "Finalizando..."
            
            extracted_data = extracted_records
            extraction_summary = {
                'total_placemarks': total_placemarks,
                'records_extracted': len(extracted_records),
                'errors_count': len(errors),
                'errors': errors[:10],  # Solo primeros 10 errores
                'file_name': kml_file.name,
                'extraction_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
            
            # Mostrar resumen de extracción
            progress_bar.value = 100
            progress_bar.description = "¡Completado!"
            
            print("✅ ¡Extracción completada exitosamente!")
            print("=" * 50)
            print(f"📁 Archivo procesado: {kml_file.name}")
            print(f"📍 Placemarks encontrados: {total_placemarks}")
            print(f"📊 Registros extraídos: {len(extracted_records)}")
            
            if errors:
                print(f"⚠️  Errores encontrados: {len(errors)}")
                if len(errors) <= 5:
                    for error in errors:
                        print(f"   • {error}")
                else:
                    for error in errors[:3]:
                        print(f"   • {error}")
                    print(f"   ... y {len(errors)-3} errores más")
            
            # Mostrar muestra de los primeros datos
            if extracted_records:
                print(f"\n📋 Muestra de los primeros registros:")
                print("-" * 30)
                
                for i, record in enumerate(extracted_records[:3]):
                    print(f"\n{i+1}. {record.get('nombre', 'Sin nombre')}")
                    if 'latitud' in record and 'longitud' in record:
                        print(f"   🌍 Coordenadas: {record['latitud']}, {record['longitud']}")
                    if 'descripcion' in record and record['descripcion']:
                        desc_preview = record['descripcion'][:100]
                        if len(record['descripcion']) > 100:
                            desc_preview += "..."
                        print(f"   📝 {desc_preview}")
                
                if len(extracted_records) > 3:
                    print(f"\n... y {len(extracted_records)-3} registros más")
            
            print(f"\n🎉 ¡Extracción finalizada!")
            print(f"💡 Procede al siguiente paso para guardar o visualizar los datos")
            
        except ET.ParseError as e:
            print(f"❌ Error parseando XML: {e}")
            print("💡 El archivo puede estar corrupto o no ser un KML válido")
        except Exception as e:
            print(f"❌ Error durante extracción: {e}")
            print(f"💡 Tipo de error: {type(e).__name__}")
        finally:
            if progress_bar.value < 100:
                progress_bar.value = 0
                progress_bar.description = "Error"

# Conectar el botón
extract_button.on_click(lambda b: extract_kml_data())

display(widgets.VBox([
    widgets.HTML("<h4>🚀 Ejecutar Extracción de Datos</h4>"),
    extract_button,
    progress_bar,
    extraction_output
]))

## 💾 Paso 7: Guardar y Exportar Resultados

Una vez que hayas extraído los datos, puedes exportarlos en diferentes formatos para su análisis:

### 📊 Formatos Disponibles:
- **CSV**: Perfecto para Excel, Google Sheets, o análisis con pandas
- **JSON**: Ideal para aplicaciones web y APIs
- **Excel**: Con formato y columnas organizadas
- **TXT**: Para revisión rápida o documentación

### 📍 Ubicaciones de Guardado:
- **Carpeta actual**: Junto al archivo KML original
- **Descargas**: En tu carpeta de descargas predeterminada
- **Personalizada**: Elige cualquier ubicación

> **💡 Tip**: Los archivos CSV son los más versátiles para análisis posterior de datos geográficos.

In [ ]:
# Exportador de datos extraídos
save_format = widgets.Dropdown(
    options=[
        ('📊 CSV (Recomendado)', 'csv'),
        ('📄 JSON', 'json'),
        ('📋 Excel (.xlsx)', 'excel'),
        ('📝 TXT (Texto plano)', 'txt'),
        ('👀 Mostrar datos (sin guardar)', 'display')
    ],
    value='csv',
    description='Formato:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

save_location = widgets.Dropdown(
    options=[
        ('📁 Misma carpeta del KML', 'same'),
        ('💾 Carpeta de Descargas', 'downloads'),
        ('🎯 Ubicación personalizada', 'custom')
    ],
    value='same',
    description='Ubicación:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

custom_path_widget = widgets.Text(
    value='',
    placeholder='C:\\MiCarpeta\\resultados\\',
    description='Ruta personalizada:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px', display='none')
)

save_filename = widgets.Text(
    value='datos_kml_extraidos',
    description='Nombre:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Botones de acción
save_button = widgets.Button(
    description='💾 Guardar Archivo',
    button_style='success',
    layout=widgets.Layout(width='150px')
)

preview_button = widgets.Button(
    description='👀 Vista Previa',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

# Output para mostrar resultados de guardado
save_output = widgets.Output()

def update_custom_path_visibility(change):
    """Mostrar/ocultar campo de ruta personalizada"""
    if change['new'] == 'custom':
        custom_path_widget.layout.display = 'block'
    else:
        custom_path_widget.layout.display = 'none'

save_location.observe(update_custom_path_visibility, names='value')

def get_save_path():
    """Determinar la ruta completa donde guardar el archivo"""
    filename = save_filename.value.strip()
    if not filename:
        filename = "datos_kml_extraidos"
    
    format_selected = save_format.value
    if format_selected != 'display':
        if format_selected == 'excel':
            extension = 'xlsx'
        else:
            extension = format_selected
        filename = f"{filename}.{extension}"
    
    location = save_location.value
    
    if location == 'same':
        # Misma carpeta del archivo KML
        kml_path = Path(kml_file_widget.value.strip())
        if kml_path.exists():
            return kml_path.parent / filename
        else:
            return Path.cwd() / filename
    
    elif location == 'downloads':
        # Carpeta de descargas del usuario
        downloads_path = Path.home() / "Downloads"
        return downloads_path / filename
    
    elif location == 'custom':
        # Ruta personalizada
        custom_path = custom_path_widget.value.strip()
        if custom_path:
            return Path(custom_path) / filename
        else:
            return Path.cwd() / filename
    
    return Path.cwd() / filename

def preview_extracted_data():
    """Mostrar vista previa de los datos extraídos"""
    with save_output:
        clear_output()
        
        if extracted_data is None:
            print("❌ No hay datos extraídos para mostrar")
            print("💡 Primero ejecuta la extracción en el Paso 6")
            return
        
        if not extracted_data:
            print("❌ Los datos extraídos están vacíos")
            return
        
        print("👀 Vista Previa de Datos Extraídos")
        print("=" * 50)
        
        # Mostrar resumen
        if extraction_summary:
            print(f"📁 Archivo: {extraction_summary.get('file_name', 'Desconocido')}")
            print(f"📊 Total registros: {extraction_summary.get('records_extracted', len(extracted_data))}")
            print(f"⏰ Extraído: {extraction_summary.get('extraction_time', 'N/A')}")
            print()
        
        # Mostrar estructura de columnas
        if extracted_data:
            columns = list(extracted_data[0].keys())
            print(f"📋 Columnas encontradas ({len(columns)}):")
            for i, col in enumerate(columns, 1):
                print(f"  {i}. {col}")
            print()
        
        # Mostrar primeros registros
        print("📊 Primeros registros:")
        print("-" * 30)
        
        for i, record in enumerate(extracted_data[:5]):
            print(f"\n📍 Registro {i+1}:")
            for key, value in record.items():
                if value:  # Solo mostrar campos con contenido
                    value_str = str(value)
                    if len(value_str) > 100:
                        value_str = value_str[:100] + "..."
                    print(f"   {key}: {value_str}")
        
        if len(extracted_data) > 5:
            print(f"\n... y {len(extracted_data)-5} registros más")
        
        print(f"\n✅ Vista previa completada")
        print(f"💡 Usa 'Guardar Archivo' para exportar todos los datos")

def save_extracted_data():
    """Guardar los datos extraídos en el formato seleccionado"""
    with save_output:
        clear_output()
        
        if extracted_data is None:
            print("❌ No hay datos para guardar")
            print("💡 Primero ejecuta la extracción en el Paso 6")
            return
        
        if not extracted_data:
            print("❌ Los datos extraídos están vacíos")
            return
        
        format_selected = save_format.value
        
        if format_selected == 'display':
            preview_extracted_data()
            return
        
        try:
            # Determinar ruta de guardado
            file_path = get_save_path()
            
            # Crear directorio si no existe
            file_path.parent.mkdir(parents=True, exist_ok=True)
            
            print(f"💾 Guardando datos en formato {format_selected.upper()}...")
            print(f"📁 Ubicación: {file_path}")
            
            # Guardar según formato
            if format_selected == 'csv':
                df = pd.DataFrame(extracted_data)
                df.to_csv(file_path, index=False, encoding='utf-8')
                print("✅ Archivo CSV guardado exitosamente")
                
            elif format_selected == 'json':
                with open(file_path, 'w', encoding='utf-8') as f:
                    json.dump({
                        'metadata': extraction_summary,
                        'data': extracted_data
                    }, f, indent=2, ensure_ascii=False)
                print("✅ Archivo JSON guardado exitosamente")
                
            elif format_selected == 'excel':
                try:
                    df = pd.DataFrame(extracted_data)
                    with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
                        df.to_excel(writer, sheet_name='Datos KML', index=False)
                        
                        # Agregar hoja de resumen si hay información
                        if extraction_summary:
                            summary_df = pd.DataFrame([extraction_summary])
                            summary_df.to_excel(writer, sheet_name='Resumen', index=False)
                    
                    print("✅ Archivo Excel guardado exitosamente")
                except ImportError:
                    print("⚠️  openpyxl no disponible. Guardando como CSV...")
                    df = pd.DataFrame(extracted_data)
                    csv_path = file_path.with_suffix('.csv')
                    df.to_csv(csv_path, index=False, encoding='utf-8')
                    print(f"✅ Archivo CSV guardado en: {csv_path}")
                
            elif format_selected == 'txt':
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write("DATOS EXTRAÍDOS DE ARCHIVO KML\n")
                    f.write("=" * 50 + "\n\n")
                    
                    if extraction_summary:
                        f.write("RESUMEN:\n")
                        for key, value in extraction_summary.items():
                            if key != 'errors':  # No incluir errores en TXT
                                f.write(f"{key}: {value}\n")
                        f.write("\n")
                    
                    f.write("DATOS:\n")
                    f.write("-" * 30 + "\n\n")
                    
                    for i, record in enumerate(extracted_data, 1):
                        f.write(f"Registro {i}:\n")
                        for key, value in record.items():
                            if value:
                                f.write(f"  {key}: {value}\n")
                        f.write("\n")
                
                print("✅ Archivo TXT guardado exitosamente")
            
            # Información adicional sobre el archivo guardado
            file_size = file_path.stat().st_size
            if file_size < 1024:
                size_str = f"{file_size} bytes"
            elif file_size < 1024 * 1024:
                size_str = f"{file_size/1024:.1f} KB"
            else:
                size_str = f"{file_size/(1024*1024):.1f} MB"
            
            print(f"📏 Tamaño del archivo: {size_str}")
            print(f"📊 Registros guardados: {len(extracted_data)}")
            
            # Sugerencias de próximos pasos
            print(f"\n🎉 ¡Archivo guardado exitosamente!")
            print(f"💡 Próximos pasos sugeridos:")
            print(f"   • Abrir el archivo en Excel o tu editor preferido")
            print(f"   • Usar los datos para análisis geográfico")
            print(f"   • Continuar con visualización en el siguiente paso")
            
        except PermissionError:
            print(f"❌ Error: No se puede escribir en {file_path}")
            print(f"💡 Verifica que no tengas el archivo abierto en otra aplicación")
            print(f"💡 O elige una ubicación diferente")
        except Exception as e:
            print(f"❌ Error guardando archivo: {e}")
            print(f"💡 Tipo de error: {type(e).__name__}")

# Conectar botones
save_button.on_click(lambda b: save_extracted_data())
preview_button.on_click(lambda b: preview_extracted_data())

display(widgets.VBox([
    widgets.HTML("<h4>💾 Guardar y Exportar Datos</h4>"),
    widgets.HBox([
        widgets.VBox([save_format, save_location, custom_path_widget]),
        widgets.VBox([save_filename, widgets.HBox([save_button, preview_button])])
    ]),
    save_output
]))

## 🗺️ Paso 8: Visualización Interactiva

¡Ahora viene la parte más emocionante! Vamos a crear mapas interactivos con tus datos extraídos.

### 🎯 Tipos de Visualización:
- **🗺️ Mapa de Puntos**: Muestra todas las ubicaciones como marcadores
- **🔥 Mapa de Calor**: Visualiza densidad de puntos por área
- **📊 Estadísticas**: Gráficos y análisis de los datos
- **📍 Mapa Agrupado**: Agrupa puntos cercanos para mejor visualización

### 🌍 Características Interactivas:
- **Zoom y navegación** libre por el mapa
- **Tooltips informativos** con datos de cada punto
- **Capas personalizables** para diferentes tipos de datos
- **Exportación** de mapas como HTML

> **💡 Tip**: Los mapas interactivos son perfectos para presentaciones y análisis exploratorio de datos geográficos.

In [ ]:
# Visualizador de mapas interactivos
map_type = widgets.Dropdown(
    options=[
        ('🗺️ Mapa de Puntos', 'scatter'),
        ('🔥 Mapa de Calor', 'heatmap'),
        ('📊 Gráfico de Distribución', 'distribution'),
        ('📍 Mapa con Clusters', 'cluster')
    ],
    value='scatter',
    description='Tipo de mapa:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

map_style = widgets.Dropdown(
    options=[
        ('🌍 Satelital', 'satellite-streets-v11'),
        ('🗺️ Calles', 'streets-v11'),
        ('🌄 Terreno', 'outdoors-v11'),
        ('⚪ Claro', 'light-v10'),
        ('⚫ Oscuro', 'dark-v10')
    ],
    value='streets-v11',
    description='Estilo:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Botón para crear mapa
create_map_button = widgets.Button(
    description='🗺️ Crear Mapa',
    button_style='primary',
    layout=widgets.Layout(width='150px')
)

# Output para mostrar el mapa
map_output = widgets.Output()

def create_interactive_map():
    """Crear mapa interactivo con los datos extraídos"""
    with map_output:
        clear_output()
        
        if extracted_data is None:
            print("❌ No hay datos para visualizar")
            print("💡 Primero ejecuta la extracción en el Paso 6")
            return
        
        # Filtrar datos con coordenadas válidas
        valid_data = []
        for record in extracted_data:
            if ('latitud' in record and 'longitud' in record and 
                record['latitud'] and record['longitud']):
                try:
                    lat = float(record['latitud'])
                    lon = float(record['longitud'])
                    if -90 <= lat <= 90 and -180 <= lon <= 180:
                        record_copy = record.copy()
                        record_copy['lat_float'] = lat
                        record_copy['lon_float'] = lon
                        valid_data.append(record_copy)
                except (ValueError, TypeError):
                    continue
        
        if not valid_data:
            print("❌ No se encontraron coordenadas válidas para mapear")
            print("💡 Verifica que los datos extraídos contengan coordenadas")
            return
        
        print(f"🗺️ Creando mapa con {len(valid_data)} puntos válidos...")
        
        try:
            if PLOTLY_AVAILABLE:
                # Crear mapa con Plotly
                selected_map_type = map_type.value
                
                if selected_map_type == 'scatter':
                    # Mapa de puntos
                    fig = go.Figure()
                    
                    # Preparar datos para hover
                    hover_text = []
                    for record in valid_data:
                        hover_info = []
                        if 'nombre' in record and record['nombre']:
                            hover_info.append(f"<b>{record['nombre']}</b>")
                        hover_info.append(f"Lat: {record['latitud']}")
                        hover_info.append(f"Lon: {record['longitud']}")
                        if 'descripcion' in record and record['descripcion']:
                            desc = record['descripcion'][:100]
                            if len(record['descripcion']) > 100:
                                desc += "..."
                            hover_info.append(f"Descripción: {desc}")
                        hover_text.append("<br>".join(hover_info))
                    
                    fig.add_trace(go.Scattermapbox(
                        lat=[r['lat_float'] for r in valid_data],
                        lon=[r['lon_float'] for r in valid_data],
                        mode='markers',
                        marker=dict(size=8, color='red'),
                        text=hover_text,
                        hovertemplate='%{text}<extra></extra>',
                        name='Ubicaciones KML'
                    ))
                    
                    # Calcular centro del mapa
                    center_lat = sum(r['lat_float'] for r in valid_data) / len(valid_data)
                    center_lon = sum(r['lon_float'] for r in valid_data) / len(valid_data)
                    
                    fig.update_layout(
                        mapbox=dict(
                            style='open-street-map',
                            center=dict(lat=center_lat, lon=center_lon),
                            zoom=10
                        ),
                        title=f"Mapa Interactivo - {len(valid_data)} Ubicaciones",
                        height=600
                    )
                    
                elif selected_map_type == 'heatmap':
                    # Mapa de calor
                    fig = go.Figure(go.Densitymapbox(
                        lat=[r['lat_float'] for r in valid_data],
                        lon=[r['lon_float'] for r in valid_data],
                        radius=10,
                        colorscale='Hot'
                    ))
                    
                    center_lat = sum(r['lat_float'] for r in valid_data) / len(valid_data)
                    center_lon = sum(r['lon_float'] for r in valid_data) / len(valid_data)
                    
                    fig.update_layout(
                        mapbox=dict(
                            style='open-street-map',
                            center=dict(lat=center_lat, lon=center_lon),
                            zoom=10
                        ),
                        title=f"Mapa de Calor - Densidad de {len(valid_data)} Puntos",
                        height=600
                    )
                    
                elif selected_map_type == 'distribution':
                    # Gráfico de distribución de coordenadas
                    fig = go.Figure()
                    
                    fig.add_trace(go.Scatter(
                        x=[r['lon_float'] for r in valid_data],
                        y=[r['lat_float'] for r in valid_data],
                        mode='markers',
                        marker=dict(
                            size=8,
                            color=[i for i in range(len(valid_data))],
                            colorscale='Viridis',
                            showscale=True
                        ),
                        text=[r.get('nombre', 'Sin nombre') for r in valid_data],
                        name='Distribución de Puntos'
                    ))
                    
                    fig.update_layout(
                        title=f"Distribución Geográfica - {len(valid_data)} Puntos",
                        xaxis_title="Longitud",
                        yaxis_title="Latitud",
                        height=600
                    )
                
                # Mostrar el mapa
                fig.show()
                
                print("✅ Mapa interactivo creado exitosamente!")
                print("💡 Puedes hacer zoom, desplazarte y hacer hover sobre los puntos")
                
                # Ofrecer guardar mapa como HTML
                save_map_html = widgets.Button(
                    description='💾 Guardar Mapa HTML',
                    button_style='info',
                    layout=widgets.Layout(width='180px')
                )
                
                def save_map_to_html(b):
                    try:
                        filename = f"mapa_kml_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
                        file_path = get_save_path().parent / filename
                        fig.write_html(file_path)
                        print(f"💾 Mapa guardado como: {file_path}")
                    except Exception as e:
                        print(f"❌ Error guardando mapa: {e}")
                
                save_map_html.on_click(save_map_to_html)
                display(save_map_html)
                
            else:
                # Fallback sin Plotly - mostrar información estadística
                print("📊 Plotly no disponible - Mostrando estadísticas:")
                print("=" * 50)
                
                lats = [r['lat_float'] for r in valid_data]
                lons = [r['lon_float'] for r in valid_data]
                
                print(f"📍 Total puntos válidos: {len(valid_data)}")
                print(f"🌍 Rango de latitud: {min(lats):.6f} a {max(lats):.6f}")
                print(f"🌐 Rango de longitud: {min(lons):.6f} a {max(lons):.6f}")
                print(f"🎯 Centro geográfico: {sum(lats)/len(lats):.6f}, {sum(lons)/len(lons):.6f}")
                
                # Mostrar distribución por cuadrantes
                center_lat = sum(lats) / len(lats)
                center_lon = sum(lons) / len(lons)
                
                cuadrantes = {'NE': 0, 'NO': 0, 'SE': 0, 'SO': 0}
                for lat, lon in zip(lats, lons):
                    if lat >= center_lat and lon >= center_lon:
                        cuadrantes['NE'] += 1
                    elif lat >= center_lat and lon < center_lon:
                        cuadrantes['NO'] += 1
                    elif lat < center_lat and lon >= center_lon:
                        cuadrantes['SE'] += 1
                    else:
                        cuadrantes['SO'] += 1
                
                print(f"\n📊 Distribución por cuadrantes:")
                for cuad, count in cuadrantes.items():
                    print(f"   {cuad}: {count} puntos ({count/len(valid_data)*100:.1f}%)")
                
                print(f"\n💡 Para mapas interactivos, instala plotly:")
                print(f"   pip install plotly")
                
        except Exception as e:
            print(f"❌ Error creando mapa: {e}")
            print(f"💡 Tipo de error: {type(e).__name__}")

# Conectar botón
create_map_button.on_click(lambda b: create_interactive_map())

display(widgets.VBox([
    widgets.HTML("<h4>🗺️ Crear Visualización Interactiva</h4>"),
    widgets.HBox([map_type, map_style]),
    create_map_button,
    map_output
]))

## 📈 Paso 9: Análisis y Estadísticas

Obtén insights valiosos de tus datos geográficos extraídos:

### 📊 Análisis Disponibles:
- **📍 Estadísticas de Ubicación**: Distribución geográfica y centros
- **📋 Análisis de Contenido**: Palabras frecuentes en descripciones
- **🗂️ Organización**: Distribución por carpetas y categorías
- **📏 Métricas Espaciales**: Distancias y agrupaciones

### 🎯 Insights Útiles:
- **Patrones geográficos** en tus datos
- **Densidad de puntos** por región
- **Calidad de datos** y completitud
- **Recomendaciones** para análisis futuro

> **💡 Tip**: Estos análisis te ayudan a entender mejor tus datos antes de tomar decisiones basadas en ellos.

In [29]:
# Analizador de datos extraídos
analysis_type = widgets.SelectMultiple(
    options=[
        ('📍 Estadísticas Geográficas', 'geographic'),
        ('📋 Análisis de Contenido', 'content'),
        ('🗂️ Distribución por Carpetas', 'folders'),
        ('📊 Calidad de Datos', 'quality'),
        ('📏 Métricas Espaciales', 'spatial'),
        ('🎯 Resumen Ejecutivo', 'summary')
    ],
    value=['geographic', 'quality', 'summary'],
    description='Análisis:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px', height='150px')
)

# Botón para ejecutar análisis
analyze_button = widgets.Button(
    description='📈 Analizar Datos',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

# Output para mostrar análisis
analysis_output = widgets.Output()

def analyze_extracted_data():
    """Realizar análisis completo de los datos extraídos"""
    with analysis_output:
        clear_output()
        
        if extracted_data is None:
            print("❌ No hay datos para analizar")
            print("💡 Primero ejecuta la extracción en el Paso 6")
            return
        
        if not extracted_data:
            print("❌ Los datos extraídos están vacíos")
            return
        
        selected_analyses = list(analysis_type.value)
        
        print("📈 Análisis de Datos KML Extraídos")
        print("=" * 60)
        
        # Preparar datos válidos
        valid_coords = []
        for record in extracted_data:
            if ('latitud' in record and 'longitud' in record and 
                record['latitud'] and record['longitud']):
                try:
                    lat = float(record['latitud'])
                    lon = float(record['longitud'])
                    if -90 <= lat <= 90 and -180 <= lon <= 180:
                        valid_coords.append((lat, lon))
                except (ValueError, TypeError):
                    continue
        
        # 1. Estadísticas Geográficas
        if 'geographic' in selected_analyses:
            print("\n🌍 ESTADÍSTICAS GEOGRÁFICAS")
            print("-" * 40)
            
            if valid_coords:
                lats = [coord[0] for coord in valid_coords]
                lons = [coord[1] for coord in valid_coords]
                
                print(f"📍 Puntos con coordenadas válidas: {len(valid_coords)}")
                print(f"🌍 Rango de latitud: {min(lats):.6f} a {max(lats):.6f}")
                print(f"🌐 Rango de longitud: {min(lons):.6f} a {max(lons):.6f}")
                
                # Centro geográfico
                center_lat = sum(lats) / len(lats)
                center_lon = sum(lons) / len(lons)
                print(f"🎯 Centro geográfico: {center_lat:.6f}, {center_lon:.6f}")
                
                # Dispersión
                lat_range = max(lats) - min(lats)
                lon_range = max(lons) - min(lons)
                print(f"📏 Dispersión latitudinal: {lat_range:.6f} grados")
                print(f"📐 Dispersión longitudinal: {lon_range:.6f} grados")
                
                # Estimación de área cubierta (aproximada)
                # 1 grado ≈ 111 km
                area_km2 = (lat_range * 111) * (lon_range * 111 * abs(cos(center_lat * 3.14159 / 180)))
                print(f"📐 Área aproximada cubierta: {area_km2:.2f} km²")
            else:
                print("❌ No se encontraron coordenadas válidas")
        
        # 2. Análisis de Contenido
        if 'content' in selected_analyses:
            print("\n📋 ANÁLISIS DE CONTENIDO")
            print("-" * 40)
            
            # Análisis de nombres
            names = [r.get('nombre', '') for r in extracted_data if r.get('nombre')]
            if names:
                print(f"🏷️  Nombres encontrados: {len(names)}")
                print(f"📏 Longitud promedio de nombres: {sum(len(n) for n in names)/len(names):.1f} caracteres")
                
                # Palabras más comunes en nombres
                from collections import Counter
                import re
                all_words = []
                for name in names:
                    words = re.findall(r'\b\w+\b', name.lower())
                    all_words.extend(words)
                
                if all_words:
                    common_words = Counter(all_words).most_common(5)
                    print("📝 Palabras más comunes en nombres:")
                    for word, count in common_words:
                        print(f"   '{word}': {count} veces")
            
            # Análisis de descripciones
            descriptions = [r.get('descripcion', '') for r in extracted_data if r.get('descripcion')]
            if descriptions:
                print(f"\n📝 Descripciones encontradas: {len(descriptions)}")
                total_chars = sum(len(d) for d in descriptions)
                print(f"📏 Total caracteres en descripciones: {total_chars:,}")
                print(f"📊 Longitud promedio: {total_chars/len(descriptions):.1f} caracteres")
                
                # Descripciones más largas y cortas
                longest = max(descriptions, key=len)
                shortest = min(descriptions, key=len)
                print(f"📏 Descripción más larga: {len(longest)} caracteres")
                print(f"📏 Descripción más corta: {len(shortest)} caracteres")
        
        # 3. Distribución por Carpetas
        if 'folders' in selected_analyses:
            print("\n🗂️  DISTRIBUCIÓN POR CARPETAS")
            print("-" * 40)
            
            folder_counts = {}
            for record in extracted_data:
                folder = record.get('carpeta', 'Sin carpeta')
                if not folder:
                    folder = 'Sin carpeta'
                folder_counts[folder] = folder_counts.get(folder, 0) + 1
            
            if len(folder_counts) > 1 or list(folder_counts.keys())[0] != 'Sin carpeta':
                print(f"📁 Carpetas encontradas: {len(folder_counts)}")
                sorted_folders = sorted(folder_counts.items(), key=lambda x: x[1], reverse=True)
                
                for folder, count in sorted_folders[:10]:  # Top 10 carpetas
                    percentage = (count / len(extracted_data)) * 100
                    print(f"   📂 {folder}: {count} elementos ({percentage:.1f}%)")
                
                if len(sorted_folders) > 10:
                    print(f"   ... y {len(sorted_folders)-10} carpetas más")
            else:
                print("📂 Todos los elementos están en la carpeta raíz")
        
        # 4. Calidad de Datos
        if 'quality' in selected_analyses:
            print("\n📊 CALIDAD DE DATOS")
            print("-" * 40)
            
            total_records = len(extracted_data)
            
            # Completitud de campos
            field_completeness = {}
            for field in ['nombre', 'latitud', 'longitud', 'descripcion']:
                complete_count = sum(1 for r in extracted_data if r.get(field) and str(r[field]).strip())
                field_completeness[field] = (complete_count, complete_count/total_records*100)
            
            print("✅ Completitud de campos principales:")
            for field, (count, percentage) in field_completeness.items():
                print(f"   {field}: {count}/{total_records} ({percentage:.1f}%)")
            
            # Validez de coordenadas
            valid_count = len(valid_coords)
            coord_validity = (valid_count / total_records) * 100
            print(f"\n🎯 Coordenadas válidas: {valid_count}/{total_records} ({coord_validity:.1f}%)")
            
            # Registros duplicados (por nombre)
            names = [r.get('nombre', '') for r in extracted_data if r.get('nombre')]
            unique_names = set(names)
            if len(names) != len(unique_names):
                duplicates = len(names) - len(unique_names)
                print(f"🔄 Posibles duplicados por nombre: {duplicates}")
            else:
                print("✅ No se detectaron duplicados por nombre")
            
            # Puntuación de calidad general
            avg_completeness = sum(p for _, p in field_completeness.values()) / len(field_completeness)
            quality_score = (avg_completeness + coord_validity) / 2
            
            if quality_score >= 80:
                quality_status = "🟢 Excelente"
            elif quality_score >= 60:
                quality_status = "🟡 Buena"
            else:
                quality_status = "🔴 Necesita mejoras"
            
            print(f"\n🏆 Puntuación de calidad general: {quality_score:.1f}% ({quality_status})")
        
        # 5. Métricas Espaciales
        if 'spatial' in selected_analyses:
            print("\n📏 MÉTRICAS ESPACIALES")
            print("-" * 40)
            
            if len(valid_coords) >= 2:
                import math
                
                def haversine_distance(lat1, lon1, lat2, lon2):
                    """Calcular distancia entre dos puntos usando fórmula de Haversine"""
                    R = 6371  # Radio de la Tierra en km
                    dlat = math.radians(lat2 - lat1)
                    dlon = math.radians(lon2 - lon1)
                    a = (math.sin(dlat/2)**2 + 
                         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * 
                         math.sin(dlon/2)**2)
                    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
                    return R * c
                
                # Calcular distancias entre puntos
                distances = []
                for i in range(len(valid_coords)):
                    for j in range(i+1, len(valid_coords)):
                        lat1, lon1 = valid_coords[i]
                        lat2, lon2 = valid_coords[j]
                        dist = haversine_distance(lat1, lon1, lat2, lon2)
                        distances.append(dist)
                
                if distances:
                    print(f"🚀 Distancia mínima entre puntos: {min(distances):.3f} km")
                    print(f"🌍 Distancia máxima entre puntos: {max(distances):.3f} km")
                    print(f"📊 Distancia promedio: {sum(distances)/len(distances):.3f} km")
                    
                    # Densidad de puntos
                    if 'geographic' in selected_analyses and area_km2 > 0:
                        density = len(valid_coords) / area_km2
                        print(f"📍 Densidad de puntos: {density:.3f} puntos/km²")
            else:
                print("❌ Se necesitan al menos 2 puntos válidos para métricas espaciales")
        
        # 6. Resumen Ejecutivo
        if 'summary' in selected_analyses:
            print("\n🎯 RESUMEN EJECUTIVO")
            print("-" * 40)
            
            print(f"📋 Dataset: {extraction_summary.get('file_name', 'Archivo KML')}")
            print(f"📊 Total de registros: {len(extracted_data)}")
            print(f"📍 Puntos georeferenciados: {len(valid_coords)}")
            
            if extraction_summary:
                print(f"⏰ Procesado: {extraction_summary.get('extraction_time', 'N/A')}")
                if extraction_summary.get('errors_count', 0) > 0:
                    print(f"⚠️  Errores durante extracción: {extraction_summary['errors_count']}")
            
            # Recomendaciones
            print(f"\n💡 RECOMENDACIONES:")
            
            if len(valid_coords) / len(extracted_data) < 0.9:
                print(f"   • Revisar y limpiar coordenadas inválidas")
            
            completeness_scores = [p for _, p in field_completeness.items()] if 'quality' in selected_analyses else []
            if completeness_scores and min(completeness_scores) < 70:
                print(f"   • Completar campos faltantes para mejor análisis")
            
            if len(valid_coords) > 100:
                print(f"   • Considerar técnicas de clustering para visualización")
            
            if len(set(r.get('carpeta', '') for r in extracted_data)) > 1:
                print(f"   • Aprovechar la organización por carpetas para análisis segmentado")
            
            print(f"   • Los datos están listos para análisis geográfico avanzado")
        
        print(f"\n" + "=" * 60)
        print(f"✅ Análisis completado - {len(selected_analyses)} módulos ejecutados")
        print(f"💡 Usa estos insights para optimizar tu análisis de datos geográficos")

# Conectar botón
analyze_button.on_click(lambda b: analyze_extracted_data())

# Importar math para cálculos espaciales
import math
from math import cos

display(widgets.VBox([
    widgets.HTML("<h4>📈 Análisis y Estadísticas de Datos</h4>"),
    analysis_type,
    analyze_button,
    analysis_output
]))

## 🎉 ¡Felicitaciones! Extracción Completada

¡Has terminado exitosamente el proceso de extracción de datos KML! 

### 🏆 Lo que has logrado:
- ✅ **Cargado y validado** tu archivo KML
- ✅ **Extraído datos completos** según tu configuración
- ✅ **Guardado resultados** en formato utilizable
- ✅ **Creado visualizaciones** interactivas
- ✅ **Analizado insights** de tus datos geográficos

### 🚀 Próximos pasos sugeridos:

#### 📊 **Para Análisis de Datos:**
- Abrir el archivo CSV en Excel o Google Sheets
- Usar pandas para análisis avanzado en Python
- Importar a herramientas GIS como QGIS

#### 🗺️ **Para Visualización:**
- Crear dashboards con Power BI o Tableau
- Integrar con aplicaciones web usando leaflet.js
- Exportar mapas para presentaciones

#### 🔄 **Para Procesamiento Continuo:**
- Guardar este notebook para futuros archivos KML
- Automatizar el proceso para múltiples archivos
- Integrar con pipelines de datos geográficos

---

### 💡 **Consejos para Optimizar tus Resultados:**

1. **🧹 Limpieza de Datos**: Si encontraste errores, revisa el archivo KML original
2. **📍 Validación**: Verifica coordenadas en Google Maps o herramientas GIS
3. **🗂️ Organización**: Aprovecha la estructura de carpetas para análisis segmentado
4. **📈 Análisis Temporal**: Si tienes fechas, considera análisis de series temporales

---

### 🆘 **¿Necesitas Ayuda?**

- **📧 Errores de extracción**: Revisa la calidad del archivo KML original
- **🗺️ Problemas de visualización**: Verifica que las coordenadas sean válidas
- **💾 Errores de guardado**: Comprueba permisos de escritura en el directorio
- **📊 Análisis avanzado**: Considera usar bibliotecas especializadas como geopandas

¡Gracias por usar el Extractor KML! 🎊

In [ ]:
# Funciones de utilidad adicionales y panel de control final
print("🎉 ¡Extractor KML Completado!")
print("=" * 50)
print("Todas las funcionalidades están listas para usar.")
print()
print("📋 RESUMEN DE FUNCIONALIDADES DISPONIBLES:")
print("1. 📦 Instalación automática de dependencias")
print("2. 📂 Carga de archivos KML desde cualquier ubicación")
print("3. 👀 Previsualización de contenido")
print("4. ⚙️ Configuración personalizable de extracción")
print("5. 🚀 Extracción completa de datos")
print("6. 💾 Exportación en múltiples formatos")
print("7. 🗺️ Visualización interactiva con mapas")
print("8. 📈 Análisis estadístico avanzado")
print()
print("💡 INSTRUCCIONES RÁPIDAS:")
print("• Ejecuta las celdas en orden secuencial")
print("• Cada paso valida automáticamente el anterior")
print("• Los resultados se guardan automáticamente")
print("• Puedes repetir cualquier paso las veces que necesites")
print()
print("🔧 SOLUCIÓN DE PROBLEMAS COMUNES:")

# Widget para mostrar/ocultar troubleshooting
troubleshooting_toggle = widgets.ToggleButton(
    value=False,
    description='🔧 Mostrar Troubleshooting',
    button_style='info',
    layout=widgets.Layout(width='250px')
)

troubleshooting_output = widgets.Output()

def show_troubleshooting(change):
    with troubleshooting_output:
        clear_output()
        if change['new']:
            print("🔧 SOLUCIÓN DE PROBLEMAS COMUNES:")
            print("=" * 45)
            print()
            print("❌ ARCHIVO NO ENCONTRADO:")
            print("   • Verifica que la ruta sea correcta")
            print("   • Usa barras invertidas dobles: C:\\\\Carpeta\\\\archivo.kml")
            print("   • O usa el botón 'Explorar KML'")
            print()
            print("❌ ERROR AL PARSEAR XML:")
            print("   • El archivo puede estar corrupto")
            print("   • Verifica que sea un archivo KML válido")
            print("   • Abre el archivo en un editor de texto para verificar")
            print()
            print("❌ NO SE ENCUENTRAN COORDENADAS:")
            print("   • Algunos KML solo tienen direcciones, no coordenadas")
            print("   • Usa herramientas de geocodificación primero")
            print("   • Verifica el formato del archivo original")
            print()
            print("❌ ERROR AL GUARDAR ARCHIVO:")
            print("   • Verifica permisos de escritura en la carpeta")
            print("   • Cierra el archivo si está abierto en Excel")
            print("   • Cambia la ubicación de guardado")
            print()
            print("❌ MAPA NO SE MUESTRA:")
            print("   • Verifica que plotly esté instalado")
            print("   • Revisa que las coordenadas sean válidas")
            print("   • Usa el análisis de calidad de datos")
            print()
            print("💡 CONSEJOS DE OPTIMIZACIÓN:")
            print("   • Archivos muy grandes (>10MB): procesa en lotes")
            print("   • Para análisis repetitivo: guarda configuraciones")
            print("   • Coordenadas inválidas: usa herramientas de validación")
            print("   • Para mapas profesionales: exporta datos a QGIS")

troubleshooting_toggle.observe(show_troubleshooting, names='value')

# Panel de estado final
status_panel = widgets.VBox([
    widgets.HTML("<h4>🎛️ Panel de Control Final</h4>"),
    widgets.HTML(f"<p><strong>📊 Estado:</strong> Notebook completo y funcional</p>"),
    widgets.HTML(f"<p><strong>⏰ Fecha:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M')}</p>"),
    widgets.HTML(f"<p><strong>🔧 Herramientas:</strong> KML Extractor GUI v1.0</p>"),
    troubleshooting_toggle,
    troubleshooting_output
])

# Botón para reiniciar todo el proceso
reset_button = widgets.Button(
    description='🔄 Reiniciar Proceso',
    button_style='warning',
    layout=widgets.Layout(width='180px')
)

def reset_extraction_process(b):
    """Reiniciar todas las variables globales"""
    global extracted_data, extraction_summary
    
    # Limpiar variables globales
    extracted_data = None
    extraction_summary = {}
    
    # Limpiar widgets principales
    kml_file_widget.value = ''
    output_filename.value = 'datos_kml_extraidos'
    
    # Resetear progress bar
    progress_bar.value = 0
    progress_bar.description = 'Listo'
    
    # Limpiar outputs
    for output_widget in [kml_validation_output, preview_output, config_output, 
                         extraction_output, save_output, map_output, analysis_output]:
        with output_widget:
            clear_output()
    
    print("🔄 Proceso reiniciado - Puedes comenzar con un nuevo archivo KML")

reset_button.on_click(reset_extraction_process)

# Botón para mostrar información del sistema
system_info_button = widgets.Button(
    description='💻 Info del Sistema',
    button_style='info',
    layout=widgets.Layout(width='180px')
)

system_info_output = widgets.Output()

def show_system_info(b):
    with system_info_output:
        clear_output()
        print("💻 INFORMACIÓN DEL SISTEMA:")
        print("=" * 35)
        print(f"🐍 Python: {sys.version.split()[0]}")
        print(f"📦 Pandas: {pd.__version__}")
        print(f"🎛️ IPywidgets: {widgets.__version__}")
        
        # Verificar bibliotecas opcionales
        try:
            import plotly
            print(f"📊 Plotly: {plotly.__version__} ✅")
        except ImportError:
            print("📊 Plotly: No instalado ❌")
        
        try:
            import openpyxl
            print(f"📋 OpenPyXL: {openpyxl.__version__} ✅")
        except ImportError:
            print("📋 OpenPyXL: No instalado ❌")
        
        # Info del workspace
        print(f"\n📁 Directorio actual: {Path.cwd()}")
        print(f"🏠 Directorio home: {Path.home()}")
        
        # Estadísticas de la sesión
        if extracted_data:
            print(f"\n📊 DATOS ACTUALES:")
            print(f"📍 Registros cargados: {len(extracted_data)}")
            print(f"📁 Archivo procesado: {extraction_summary.get('file_name', 'N/A')}")

system_info_button.on_click(show_system_info)

display(status_panel)
display(widgets.HBox([reset_button, system_info_button]))
display(system_info_output)

print("🚀 ¡El Extractor KML está completamente funcional!")
print("💡 Comienza con el Paso 1 o usa los botones de control de arriba.")

🎉 ¡Extractor KML Completado!
Todas las funcionalidades están listas para usar.

📋 RESUMEN DE FUNCIONALIDADES DISPONIBLES:
1. 📦 Instalación automática de dependencias
2. 📂 Carga de archivos KML desde cualquier ubicación
3. 👀 Previsualización de contenido
4. ⚙️ Configuración personalizable de extracción
5. 🚀 Extracción completa de datos
6. 💾 Exportación en múltiples formatos
7. 🗺️ Visualización interactiva con mapas
8. 📈 Análisis estadístico avanzado

💡 INSTRUCCIONES RÁPIDAS:
• Ejecuta las celdas en orden secuencial
• Cada paso valida automáticamente el anterior
• Los resultados se guardan automáticamente
• Puedes repetir cualquier paso las veces que necesites

🔧 SOLUCIÓN DE PROBLEMAS COMUNES:


Output()

🚀 ¡El Extractor KML está completamente funcional!
💡 Comienza con el Paso 1 o usa los botones de control de arriba.


In [31]:
# 🔧 Validación del Fix - Prueba con BMW.kml
print("🔧 Validando corrección del error de extracción...")
print("=" * 50)

# Probar rápidamente con el archivo BMW.kml
test_file = "BMW.kml"
test_path = Path(test_file)

if test_path.exists():
    try:
        print(f"📁 Probando con archivo: {test_file}")
        
        # Intentar parsear
        tree = ET.parse(test_path)
        root = tree.getroot()
        print("✅ Archivo parseado correctamente")
        
        # Detectar namespace
        namespace = ''
        if root.tag.startswith('{'):
            namespace = root.tag.split('}')[0] + '}'
        print(f"🏷️ Namespace detectado: {namespace if namespace else 'Sin namespace'}")
        
        # Buscar placemarks
        placemarks = root.findall('.//' + namespace + 'Placemark')
        print(f"📍 Placemarks encontrados: {len(placemarks)}")
        
        # Probar primer placemark
        if placemarks:
            first_placemark = placemarks[0]
            
            # Buscar coordenadas
            coords_elem = first_placemark.find('.//' + namespace + 'coordinates')
            if coords_elem is not None and coords_elem.text:
                coords = coords_elem.text.strip()
                print(f"🌍 Primera coordenada: {coords}")
            
            # Buscar dirección
            addr_elem = first_placemark.find(namespace + 'address')
            if addr_elem is not None and addr_elem.text:
                print(f"📍 Primera dirección: {addr_elem.text}")
            
        print("✅ Prueba básica exitosa - El fix funciona!")
        print("💡 Ahora puedes usar el extractor normalmente")
        
    except Exception as e:
        print(f"❌ Error en prueba: {e}")
        print(f"💡 Tipo de error: {type(e).__name__}")
else:
    print(f"⚠️ Archivo {test_file} no encontrado en el directorio actual")
    print(f"📂 Directorio actual: {Path.cwd()}")
    print("💡 Asegúrate de que el archivo BMW.kml esté en la misma carpeta")

print("\n🎯 Estado del Fix:")
print("✅ Corregido tree.getRoot() → tree.getroot()")
print("✅ Corregido manejo de carpetas/folders")
print("✅ Mejorado manejo de errores en extracción")
print("\n💡 El extractor ahora debería funcionar correctamente")

🔧 Validando corrección del error de extracción...
📁 Probando con archivo: BMW.kml
✅ Archivo parseado correctamente
🏷️ Namespace detectado: {http://www.opengis.net/kml/2.2}
📍 Placemarks encontrados: 68
🌍 Primera coordenada: -81.5380936,31.122383,0
📍 Primera dirección: Joe Franks Harris Blvd 106 Brunswick 31523
✅ Prueba básica exitosa - El fix funciona!
💡 Ahora puedes usar el extractor normalmente

🎯 Estado del Fix:
✅ Corregido tree.getRoot() → tree.getroot()
✅ Corregido manejo de carpetas/folders
✅ Mejorado manejo de errores en extracción

💡 El extractor ahora debería funcionar correctamente
